## <i> Audio data preprocessing

In [ ]:
# Audio Example Preprocessing Script
import os
import sys
import pickle as pkl
from pathlib import Path
import numpy as np
from utils import vggish_input, params

# Configuration
PROJECT_ROOT = Path.cwd()
sys.path.append(str(PROJECT_ROOT / ".." / ".." / ".." / "HCAR"))

# Set paths
DATA_PATH = Path("../../Data/Train_Data/2_TrainingDataset")
SAVE_PATH = Path("../../Data/Train_Data/4_AudioExamples")

# Audio preprocessing settings
participant_list = [p.name for p in DATA_PATH.iterdir() if p.is_dir()]
hand = 'Right'
sub_sr = 16000  # Subsample rate

# Ensure save path exists
SAVE_PATH.mkdir(parents=True, exist_ok=True)

# Process each participant's data
for participant in participant_list:
    hand_path = DATA_PATH / participant / hand
    print(f"Processing participant: {participant}, hand: {hand}")

    example_shapes = []

    # Iterate over all pickle files
    for pickle_path in sorted(hand_path.iterdir()):
        if pickle_path.suffix.lower() != '.pkl':
            continue

        print(f"---- {pickle_path.name} ----")

        # Load multimodal pickle
        with open(pickle_path, 'rb') as f:
            fdata = pkl.load(f)

        og_audio = fdata.get('Audio')
        if og_audio is None:
            print(f"No 'Audio' key in {pickle_path.name}, skipping.")
            continue

        print(f"Original Audio shape: {og_audio.shape}")

        # Prepare save directory per participant and rate
        save_dir = SAVE_PATH / participant / hand / str(sub_sr)
        save_dir.mkdir(parents=True, exist_ok=True)

        # Subsample audio
        factor = int(16000 // sub_sr)
        wav_data = og_audio[::factor].astype(np.int16)

        # Generate VGGish examples in 10s chunks
        chunk_size = sub_sr * 10
        chunks = [wav_data[i:i + chunk_size] for i in range(0, len(wav_data), chunk_size)]
        audio_examples_list = []

        for chunk in chunks:
            examples = vggish_input.wavform_to_examples(
                chunk,
                lower_edge_hertz=params.get('mel_lower_edge_hertz', 10),
                upper_edge_hertz=params.get('mel_upper_edge_hertz', sub_sr // 2),
                sr=sub_sr
            )
            audio_examples_list.append(examples)

        # Concatenate and record shape
        audio_examples = np.concatenate(audio_examples_list, axis=0)
        example_shapes.append(audio_examples.shape)
        print(f"Generated examples shape: {audio_examples.shape}")

        # Save to pickle
        out_file = save_dir / pickle_path.name
        with open(out_file, 'wb') as f:
            pkl.dump(audio_examples, f)
        print(f"Saved: {out_file}")

    # Check consistency
    if len(set(example_shapes)) > 1:
        print(f"Inconsistent example shapes: {example_shapes}")
